# INDIAN START UP ECOSYSTEM

DESCRIPTIPN

In this project,we analyze and visualize industry data and identify trends and opportunities in the Indian startup market. The analysis will cover funding trends, the geographic distribution of the start-ups,funding sources, and the industrial sector in which the start-ups operate. The insights gained from this project will help venture capitalists stay ahead of the curve and identify promising investment opportunities.

<b>QUESTIONS<b>

1. Which sector or industry received  the least and most funding in each year(2018-2021).
2. Which investor provided the least and highest funding in each year(2018-2021).
3. Which Company received least and more Funds in each year (2018-2021).
4. The location received the highest and least funding in the years under review.
5. Does the Locaction of the startup impact the investments.

In [217]:
%pip install pyodbc  
%pip install python-dotenv 
%pip install summarytools
%pip install scipy




[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [218]:
# importing the necessary libraries
 
import pyodbc # Database Connectivity with  Open Database Connectivity (pyodbc)
 
from dotenv import dotenv_values # dotenv library to load environment variables
# pandas for data manipulation and analysis
import pandas as pd
import numpy as np
import warnings
 
warnings.filterwarnings('ignore')
import seaborn as sns
import matplotlib.pyplot as plt
from summarytools import dfSummary
from scipy.stats import f_oneway
# show floats to only 4 deicmal places
pd.set_option('display.float_format', lambda x: '%0.4f' % x)

In [219]:
# Load environment variables from .env file into a dictionary
environment_variables = dotenv_values('.env')


# Get the values for the credentials you set in the '.env' file
server = environment_variables.get("SERVER")
database = environment_variables.get("DATABASE")
username = environment_variables.get("UID")
password = environment_variables.get("PWD")


connection_string = f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password};MARS_Connection=yes;MinProtocolVersion=TLSv1.2;"


In [220]:
# Use the connect method of the pyodbc library and pass in the connection string.
# This will connect to the server and might take a few seconds to be complete. 
# Check your internet connection if it takes more time than necessary

connection = pyodbc.connect(connection_string)

In [221]:
# Load the dataset SQL query
query1 = "Select * from dbo.LP1_startup_funding2020"
query2 = "Select * from dbo.LP1_startup_funding2021"

<b>LOADING THE DATASETS<b>

In [222]:
# Read data from the  on dataset
data_2018 = pd.read_csv('dataset\startup_funding2018.csv')
data_2019 = pd.read_csv('dataset\startup_funding2019.csv')
data_2020 = pd.read_sql(query1, connection)
data_2021 = pd.read_sql(query2, connection)
 

EXPLORATORY DATA ANALYSIS-EDA

-Here we inspect the datasets in depth year by year and column by column. This is to clean it and make it ready for analysis.


In [223]:
data_2018.head()

,Company Name,Industry,Round/Series,Amount,Location,About Company
0,TheCollegeFever,"Brand Marketing, Event Promotion, Marketing, S...",Seed,250000,"Bangalore, Karnataka, India","TheCollegeFever is a hub for fun, fiesta and f..."
1,Happy Cow Dairy,"Agriculture, Farming",Seed,"₹40,000,000","Mumbai, Maharashtra, India",A startup which aggregates milk from dairy far...
2,MyLoanCare,"Credit, Financial Services, Lending, Marketplace",Series A,"₹65,000,000","Gurgaon, Haryana, India",Leading Online Loans Marketplace in India
3,PayMe India,"Financial Services, FinTech",Angel,2000000,"Noida, Uttar Pradesh, India",PayMe India is an innovative FinTech organizat...
4,Eunimart,"E-Commerce Platforms, Retail, SaaS",Seed,—,"Hyderabad, Andhra Pradesh, India",Eunimart is a one stop solution for merchants ...


<h>SPECIFIC ISSUES WITH 2018 DATASET<h>


-The location column lists the city, state and country name. This would have to be stripped to only the city name.

-2018 dataset does not have a column for investors.

-Names of some of the columns has to be changed to be consistent with those of 2019,2020 and 2021.

-Create a unique column for the year this is to make subsequent analysis easier.

-Drop the column 'About Company' it has no bearing on the subsequent analysis.

-Inspect each column separately to identify its own perculiar issues.

-In the amount column a couple of entries are in Indian Rupees. These have to be converted to US Dollars using the appriopriate exchange rate for the period under review.

In [224]:
#Create a unique year column
data_2018['Year']='2018'

In [225]:
# Drop the [About Company] column

del data_2018['About Company']

In [226]:
# Harmonizing the 2018 dataset columns to match the other three datasets
data_2018.columns=['Company_Brand','Sector','Stage','Amount($)','HeadQuarter','Year']

data_2018.head()

,Company_Brand,Sector,Stage,Amount($),HeadQuarter,Year
0,TheCollegeFever,"Brand Marketing, Event Promotion, Marketing, S...",Seed,250000,"Bangalore, Karnataka, India",2018
1,Happy Cow Dairy,"Agriculture, Farming",Seed,"₹40,000,000","Mumbai, Maharashtra, India",2018
2,MyLoanCare,"Credit, Financial Services, Lending, Marketplace",Series A,"₹65,000,000","Gurgaon, Haryana, India",2018
3,PayMe India,"Financial Services, FinTech",Angel,2000000,"Noida, Uttar Pradesh, India",2018
4,Eunimart,"E-Commerce Platforms, Retail, SaaS",Seed,—,"Hyderabad, Andhra Pradesh, India",2018


CREATE UNIQUE SECTOR AND LOCATION (CITY) COLUMNS

In [227]:
# Split the 'Sector' column
df_18 = data_2018['Sector'].str.split(pat=',', n=1, expand=True)

# Create a new column 'industry1' based on the first part of the split
data_2018['industry1'] = df_18[0]

#Drop the 'Original' Sector Column
data_2018.drop('Sector', axis=1, inplace=True)

# Rename [industry1] Column as 'New' [Sector] Column

data_2018=data_2018.rename(columns={'industry1':'Sector'})


In [228]:
# Create a new location column by splitting the HeadQuarter Column

df_2018=data_2018['HeadQuarter'].str.split(pat=',', n=1, expand=True)
data_2018['location']=df_2018[0]

# Drop the 'Old' HeadQuarter Column

data_2018.drop('HeadQuarter', axis=1, inplace=True)

# Rename the 'New' Location column as HeadQuarter
data_2018=data_2018.rename(columns={'location':'HeadQuarter'})


In [229]:
# Get the count of each unique value in the 'Category' column
value_counts_C = data_2018['Company_Brand'].value_counts()
value_counts_S = data_2018['Stage'].value_counts()
value_counts_A = data_2018['Amount($)'].value_counts()
value_counts_Y = data_2018['Year'].value_counts()
value_counts_Se = data_2018['Sector'].value_counts()
value_counts_H = data_2018['HeadQuarter'].value_counts()


# Display the count of each unique value
print("Value Counts in 'Company_Brand':")
#print("Value Counts in 'Category':")
#print("Value Counts in 'Category':")
#print("Value Counts in 'Category':")
#print("Value Counts in 'Category':")
#print("Value Counts in 'Category':")
print(value_counts_C)


Value Counts in 'Company_Brand':
TheCollegeFever                                                  2
NIRAMAI Health Analytix                                          1
Drivezy                                                          1
Hush - Speak Up. Make Work Better                                1
The Souled Store                                                 1
Perspectico                                                      1
Kogta Financial India Limited                                    1
Hospals                                                          1
UrbanClap                                                        1
Square Off                                                       1
Gaia Smart Cities                                                1
ERUDITUS Executive Education                                     1
Career Anna                                                      1
Wicked Ride Adventure Services Private Limited                   1
Cricnwin                     

CHECK FOR DUPLICATES IN THE 2018 DATASET

In [230]:
df_2018.duplicated().value_counts()

True     476
False     50
dtype: int64

DROP THE DUPLICATE ENTRIES WHILST MAINTAINING THE INTEGRITY OF THE ORIGINAL DATAFRAME

In [231]:
df_2018.drop_duplicates(keep='first', inplace=True)

CONFIRM THERE ARE NO DUPLICATE ENTRIES IN THE DATAFRAME df_startup

In [232]:
df_2018.duplicated().value_counts()

False    50
dtype: int64

CURRENCY CONVERSION (Rupees to US Dollars)

In [233]:
#2018 Exchange rate Rupee to a Dollar.
exchange_rate = 68.14

def rupee_to_dollar(amount):
    if isinstance(amount, str) and amount.startswith('₹'):
        amount = float(amount.replace(',', '')[1:]) * exchange_rate
        return f'${amount:.2f}'
    else:
        return amount

# remove rupee sign and comma, and convert to dollar equivalent
data_2018['Amount($)'] = data_2018['Amount($)'].apply(lambda x: rupee_to_dollar(x))

In [234]:
data_2018.head()

,Company_Brand,Stage,Amount($),Year,Sector,HeadQuarter
0,TheCollegeFever,Seed,250000,2018,Brand Marketing,Bangalore
1,Happy Cow Dairy,Seed,$2725600000.00,2018,Agriculture,Mumbai
2,MyLoanCare,Series A,$4429100000.00,2018,Credit,Gurgaon
3,PayMe India,Angel,2000000,2018,Financial Services,Noida
4,Eunimart,Seed,—,2018,E-Commerce Platforms,Hyderabad


<h>SPECIFIC ISSUES WITH 2019 DATASET<h>

- The following three columns will be dropped as they have no bearing on the analysis to be performed subsequently-[Founded],[What it does],[Founders].

- A unique year only column has be created to enable year on year analysis to be performed.

- Change the Company/Brand column name to Company_Brand.


DROP COLUMNS THAT HAVE NO BEARING ON THE OVERALL ANALYSIS

In [235]:
# Drop the [Founded, What it does, Founders ] column

columns_to_drop = ['Founded', 'What it does', 'Founders']
data_2019.drop(columns=columns_to_drop, inplace=True)


CREATE A UNIQUE YEAR COLUMN FOR 2019 DATASET

In [236]:
# Create a unique year column
data_2019['Year']='2019'

HARMONISE THE NAMES OF THE COLUMNS; Company/Brand to Company_Brand

In [237]:
# Change the Company/Brand column name to Company_Brand
 
data_2019.rename(columns={'Company/Brand': 'Company_Brand'})


,Company_Brand,HeadQuarter,Sector,Investor,Amount($),Stage,Year
0,Bombay Shaving,NaN,Ecommerce,Sixth Sense Ventures,"$6,300,000",NaN,2019
1,Ruangguru,Mumbai,Edtech,General Atlantic,"$150,000,000",Series C,2019
2,Eduisfun,Mumbai,Edtech,"Deepak Parekh, Amitabh Bachchan, Piyush Pandey","$28,000,000",Fresh funding,2019
3,HomeLane,Chennai,Interior design,"Evolvence India Fund (EIF), Pidilite Group, FJ...","$30,000,000",Series D,2019
4,Nu Genes,Telangana,AgriTech,Innovation in Food and Agriculture (IFA),"$6,000,000",NaN,2019
5,FlytBase,Pune,Technology,Undisclosed,Undisclosed,NaN,2019
6,Finly,Bangalore,SaaS,"Social Capital, AngelList India, Gemba Capital...",Undisclosed,NaN,2019
7,Kratikal,Noida,Technology,"Gilda VC, Art Venture, Rajeev Chitrabhanu.","$1,000,000",Pre series A,2019
8,Quantiphi,NaN,AI & Tech,Multiples Alternate Asset Management,"$20,000,000",Series A,2019
9,Lenskart,Delhi,E-commerce,SoftBank,"$275,000,000",Series G,2019


In [238]:
data_2019.head()

,Company/Brand,HeadQuarter,Sector,Investor,Amount($),Stage,Year
0,Bombay Shaving,NaN,Ecommerce,Sixth Sense Ventures,"$6,300,000",NaN,2019
1,Ruangguru,Mumbai,Edtech,General Atlantic,"$150,000,000",Series C,2019
2,Eduisfun,Mumbai,Edtech,"Deepak Parekh, Amitabh Bachchan, Piyush Pandey","$28,000,000",Fresh funding,2019
3,HomeLane,Chennai,Interior design,"Evolvence India Fund (EIF), Pidilite Group, FJ...","$30,000,000",Series D,2019
4,Nu Genes,Telangana,AgriTech,Innovation in Food and Agriculture (IFA),"$6,000,000",NaN,2019


<h>SPECIFIC ISSUES WITH 2020 DATASET<h>

- The following three columns will be dropped as they have no bearing on the analysis to be performed subsequently-[Founded],[What it does],[Founders],[column10]

- A unique year only column has be created to enable year on year analysis to be performed.


In [239]:
# Drop the [Founded, What it does, Founders ] column

columns_to_drop = ['Founded', 'What_it_does', 'Founders','column10']
data_2020.drop(columns=columns_to_drop, inplace=True)

CREATE A UNIQUE YEAR COLUMN FOR 2020 DATASET

In [240]:
# Create a unique year column

data_2020['Year']='2020'

In [241]:
data_2020.head()

,Company_Brand,HeadQuarter,Sector,Investor,Amount,Stage,Year
0,Aqgromalin,Chennai,AgriTech,Angel investors,200000.0000,None,2020
1,Krayonnz,Bangalore,EdTech,GSF Accelerator,100000.0000,Pre-seed,2020
2,PadCare Labs,Pune,Hygiene management,Venture Center,NaN,Pre-seed,2020
3,NCOME,New Delhi,Escrow,"Venture Catalysts, PointOne Capital",400000.0000,None,2020
4,Gramophone,Indore,AgriTech,"Siana Capital Management, Info Edge",340000.0000,None,2020


<h>SPECIFIC ISSUES WITH 2020 DATASET<h>

- The following three columns will be dropped as they have no bearing on the analysis to be performed subsequently-[Founded],[What_it_does],[Founders].

- A unique year only column has be created to enable year on year analysis to be performed.

- Change [Amount] to [Amount($)].

In [242]:
# Drop the [Founded, What it does, Founders ] column

columns_to_drop = ['Founded', 'What_it_does', 'Founders']
data_2021.drop(columns=columns_to_drop, inplace=True)

In [243]:
#  Create a unique year column

data_2021['Year']='2021'

In [244]:
data_2021.rename(columns={'Amount': 'Amount($)'})

,Company_Brand,HeadQuarter,Sector,Investor,Amount($),Stage,Year
0,Unbox Robotics,Bangalore,AI startup,"BEENEXT, Entrepreneur First","$1,200,000",Pre-series A,2021
1,upGrad,Mumbai,EdTech,"Unilazer Ventures, IIFL Asset Management","$120,000,000",None,2021
2,Lead School,Mumbai,EdTech,"GSV Ventures, Westbridge Capital","$30,000,000",Series D,2021
3,Bizongo,Mumbai,B2B E-commerce,"CDC Group, IDG Capital","$51,000,000",Series C,2021
4,FypMoney,Gurugram,FinTech,"Liberatha Kallat, Mukesh Yadav, Dinesh Nagpal","$2,000,000",Seed,2021
5,Urban Company,New Delhi,Home services,Vy Capital,"$188,000,000",None,2021
6,Comofi Medtech,Bangalore,HealthTech,"CIIE.CO, KIIT-TBI","$200,000",None,2021
7,Qube Health,Mumbai,HealthTech,Inflection Point Ventures,Undisclosed,Pre-series A,2021
8,Vitra.ai,Bangalore,Tech Startup,Inflexor Ventures,Undisclosed,None,2021
9,Taikee,Mumbai,E-commerce,None,"$1,000,000",None,2021


In [245]:
data_2021.head()

,Company_Brand,HeadQuarter,Sector,Investor,Amount,Stage,Year
0,Unbox Robotics,Bangalore,AI startup,"BEENEXT, Entrepreneur First","$1,200,000",Pre-series A,2021
1,upGrad,Mumbai,EdTech,"Unilazer Ventures, IIFL Asset Management","$120,000,000",None,2021
2,Lead School,Mumbai,EdTech,"GSV Ventures, Westbridge Capital","$30,000,000",Series D,2021
3,Bizongo,Mumbai,B2B E-commerce,"CDC Group, IDG Capital","$51,000,000",Series C,2021
4,FypMoney,Gurugram,FinTech,"Liberatha Kallat, Mukesh Yadav, Dinesh Nagpal","$2,000,000",Seed,2021


<b>MERGE THE FOUR (4) DATASETS TO PERFORM THE FINAL CLEANING AND ANALYSIS<b>

The four datasets have been combined into a single dataFrame called df_startup. 

The individual columns in this new DataFrame will inspected and cleaned. 

Only six relevant columns are in this new DataFrame, 
they are Company/Brand,Stage,Amount,Sector,Headquarter and Investor.

In [246]:
df_startup=pd.concat([data_2018,data_2019,data_2020,data_2021],ignore_index=True)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [247]:
df_startup.head()

,Company_Brand,Stage,Amount($),Year,Sector,HeadQuarter,Company/Brand,Investor,Amount
0,TheCollegeFever,Seed,250000,2018,Brand Marketing,Bangalore,NaN,NaN,NaN
1,Happy Cow Dairy,Seed,$2725600000.00,2018,Agriculture,Mumbai,NaN,NaN,NaN
2,MyLoanCare,Series A,$4429100000.00,2018,Credit,Gurgaon,NaN,NaN,NaN
3,PayMe India,Angel,2000000,2018,Financial Services,Noida,NaN,NaN,NaN
4,Eunimart,Seed,—,2018,E-Commerce Platforms,Hyderabad,NaN,NaN,NaN


In [248]:
# Drop the Company/Brand and Amount columns

df_startup.drop(['Company/Brand','Amount'], axis=1, inplace=True)


In [249]:
df_startup.head()

,Company_Brand,Stage,Amount($),Year,Sector,HeadQuarter,Investor
0,TheCollegeFever,Seed,250000,2018,Brand Marketing,Bangalore,NaN
1,Happy Cow Dairy,Seed,$2725600000.00,2018,Agriculture,Mumbai,NaN
2,MyLoanCare,Series A,$4429100000.00,2018,Credit,Gurgaon,NaN
3,PayMe India,Angel,2000000,2018,Financial Services,Noida,NaN
4,Eunimart,Seed,—,2018,E-Commerce Platforms,Hyderabad,NaN


<b>WE PERFORM A DETAILED COLUMN BY COLUMN CLEANING OF THE NEW DATAFRAME df_startup<b>

<h>ISSUES IDENTIFIED WITH THE NEW DATAFRAME THAT NEEDS TO BE ADDRESSED<h>


-The new DataFrame df_startup has 33 duplicate entries.

-The Amount column is presented in an object datatype this has to be transformed into a float.

-The year column is an object data type this has to be changed to a datetime data type.

-In the Stage column some of the stages are spelt slighty differently, these need to be harmonised and made consistent.

-In the Sector column, same sectors are spelt differently for different companies.

In [250]:
# CHECK FOR DUPLICATES

df_startup.duplicated().value_counts()

False    2846
True       33
dtype: int64

In [251]:
# DROP THE DUPLICATES ENTRIES WHILST MAINTAINING THE INTEGRITY OF THE ORIGINAL DATAFRAME

df_startup.drop_duplicates(keep='first', inplace=True)

In [252]:
df_startup.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2846 entries, 0 to 2878
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Company_Brand  2757 non-null   object
 1   Stage          1922 non-null   object
 2   Amount($)      614 non-null    object
 3   Year           2846 non-null   object
 4   Sector         2828 non-null   object
 5   HeadQuarter    2732 non-null   object
 6   Investor       2222 non-null   object
dtypes: object(7)
memory usage: 177.9+ KB


In [253]:
# Convert the Amount column from Object data type to Float data type.

df_startup['Amount($)'] = df_startup['Amount($)'].str.replace('$', '').str.replace(',', '').replace(['—', 'Undisclosed','Undiclosed','Undiclsosed','Undislosed'], np.nan).astype(float)

In [254]:
# Convert the year from Object to its appriopriate data type
df_startup['Year']=pd.to_datetime(df_startup.Year)


In [255]:
#df_startup['Year'] = df_startup['Year'].dt.year

In [256]:
df_startup.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2846 entries, 0 to 2878
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Company_Brand  2757 non-null   object        
 1   Stage          1922 non-null   object        
 2   Amount($)      454 non-null    float64       
 3   Year           2846 non-null   datetime64[ns]
 4   Sector         2828 non-null   object        
 5   HeadQuarter    2732 non-null   object        
 6   Investor       2222 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(5)
memory usage: 177.9+ KB
